# [【SOTA】マイナビ × SIGNATE Student Cup 2019: 賃貸物件の家賃予測](https://signate.jp/competitions/264)

In [43]:
import pandas as pd
import pathlib
import os

# 学習データ、テストデータの読み込み
train_path = pathlib.Path("./DATA/train.csv")
test_path = pathlib.Path("./DATA/test.csv")

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

In [44]:
# データ前処理
train_data = train_data.drop(["id"], axis=1)
test_data = test_data.drop(["id"], axis=1)

# 特徴データと目的変数の設定
train_x = train_data.drop(["賃料"], axis=1)
train_y = train_data["賃料"]

test_x = test_data

In [45]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import KFold

scores = []
# GBDT用のハイパーパラメータ
prams = {"objective": "binary:logistic", "sileng":1, "random_state":71}
num_round = 50

# クロスバリデーション
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx, in kf.split(train_x):
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    
    # モデルの学習
    model = XGBClassifier(n_estimators=20, random_state=71)
    model.fit(tr_x, tr_y)
    
    # バリデーションデータの予測値を確立で出力
    va_pred = model.predict_proba(va_x)[:, 1]
    
    # バリデーションデータでのスコア計算
    logloss = log_loss(va_y, va_pred)
    accuracy = accuracy_score(va_y, va_pred > 0.5)
    
    # foldスコアの保存
    scores_logloss.append(logloss)
    scores_accuracy.append(accuracy)

# 各foldのスコア平均を出力
logloss = np.mean(scores_logloss)
accuracy = np.mean(scores_accuracy)
print(f'logloss: {logloss:.4f}, accuracy: {accuracy:.4f}')
    
    # GBDTで学習実行
    #dtrain = xgb.DMatrix(tr_x, label=tr_y) # enable_categorical=True
    #dvalid = xgb.DMatrix(va_x, lavel=va_y)
    # dtest = xgb.DMatrix(test_x)
    
    #watchlist = [(dtrain, "train"), (dvalid, "eval")]
    #model = xgb.train(params, dtrain, num_round, evals=watchlist)
    
    # バリデーションデータでのスコアの確認
    #va_pred = model.predict(dvalid)
    #score = log_loss(va_y, va_pred)
    #print(f'logloss: {score:.4f}')
  
# テストデータでの予測
#pred = model.predict(dtest)

ValueError: DataFrame.dtypes for data must be int, float, bool or categorical.  When
                categorical type is supplied, DMatrix parameter
                `enable_categorical` must be set to `True`.所在地, アクセス, 間取り, 築年数, 方角, 面積, 所在階, バス・トイレ, キッチン, 放送・通信, 室内設備, 駐車場, 周辺環境, 建物構造, 契約期間

In [57]:
group = test_data.groupby("所在階")
print(group.size())

所在階
10階／10階建          113
10階／10階建（地下1階）      1
10階／10階建（地下2階）      1
10階／11階建           86
10階／11階建（地下1階）      3
                 ... 
／6階建（地下1階）          2
／7階建                6
／7階建（地下1階）          1
／8階建                3
／9階建                1
Length: 1020, dtype: int64


In [60]:
test = test_data["所在階"].str.split("／")

In [69]:
test.head()

0    [8階, 8階建]
1    [3階, 4階建]
2    [1階, 4階建]
3    [1階, 2階建]
4    [4階, 4階建]
Name: 所在階, dtype: object

In [74]:
test[0][1]

'8階建'

In [82]:
test_data["間取り"].value_counts()

1K            11731
1R             5095
1LDK           4879
2LDK           2579
1DK            1889
2DK            1806
3LDK           1116
2K              887
3DK             432
1LDK+S(納戸)      249
2LDK+S(納戸)      169
4LDK            126
3LDK+S(納戸)       65
1K+S(納戸)         55
3K               54
5LDK             23
4DK              17
1DK+S(納戸)        13
2DK+S(納戸)        13
4LDK+S(納戸)       12
4K                8
6LDK              7
5K                6
5LDK+S(納戸)        5
6LDK+S(納戸)        4
5DK               4
3DK+S(納戸)         4
2K+S(納戸)          2
4DK+S(納戸)         2
5DK+S(納戸)         2
6DK               2
8LDK              1
3K+S(納戸)          1
11R               1
6K                1
1LK               1
2R                1
Name: 間取り, dtype: int64